In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
url='https://query1.finance.yahoo.com/v7/finance/download/%5ENSEI?period1=1559692800&period2=1591315200&interval=1d&events=history'
df=pd.read_csv(url, header=0,  encoding = "ISO-8859-1", parse_dates=True, index_col='Date').copy()## header=0 takes the first line as column headers
pd.options.display.max_columns=None
pd.options.display.max_rows=None
df= df.dropna(how='any') #to drop NaN rows and columns
df

In [ ]:
df['EMA12'] = df['Close'].ewm(span=12).mean()
df['EMA26'] = df['Close'].ewm(span=26).mean()
df['MACD']=df['EMA12']-df['EMA26']
df['EMA9M']=df['MACD'].ewm(span=9).mean() #trigger line - 9 day EMA of the MACD

In [ ]:
df['MACD'].plot(figsize=(17,8), legend = True)
df['EMA9M'].plot(figsize=(17,8), legend = True)
#df['Close'].plot(legend = True)
plt.axhline(y=0, color='r', linestyle='-')
plt.grid(True);

In [ ]:
df['prevMACD']=df['MACD'].shift(1)
df['prevEMA9M']=df['EMA9M'].shift(1)
df['Bull']=(df['MACD']>df['EMA9M']) & (df['prevEMA9M']>=df['prevMACD'])
df['Bear']=(df['MACD']<df['EMA9M']) & (df['prevEMA9M']<=df['prevMACD'])
df['TriggerIndicator']=[1 if df.loc[ei,'Bull']|df.loc[ei,'Bear'] 
                        else 0 for ei in df.index]
triggercrossover=df['TriggerIndicator']==1
df


In [ ]:
df['MACD'].plot(figsize=(17,8), legend = True)
df['EMA9M'].plot(figsize=(17,8), legend = True)
#df['Close'].plot(legend = True)
plt.axhline(y=0, color='r', linestyle='-',label='CentreLine')
plt.grid(True);
plt.title('Trigger line Indicator')
df['Bullish']=[100 if df.loc[ei,'Bull'] else 0 for ei in df.index]
df['Bearish']=[-100 if df.loc[ei,'Bear'] else 0 for ei in df.index]
df['Bullish'].plot(figsize=(17,8),legend=True, color='green')
df['Bearish'].plot(figsize=(17,8),legend=True, color='red')

Including Centreline Crossovers:  
A risk free buy signal to be generated only if MACD>0 and MACD>EMA9M  
A risk free sell signal only when MACD<0 and MACD<EMA9M

In [ ]:
df['Buy']= [150 if (df.loc[ei,'Bullish'] and df.loc[ei,'MACD']>0) else 0 for ei in df.index]
df['Sell']= [-150 if (df.loc[ei,'Bearish'] and df.loc[ei,'MACD']<0) else 0 for ei in df.index]

df['Buy'].plot(figsize=(17,8),legend=True, color='green',label='BuyRiskFree')
df['Sell'].plot(figsize=(17,8),legend=True, color='red',label='SellRiskFree')

df['Bullish'].plot(figsize=(17,8),legend=True, color='grey',label='ShortTermBuySignal')
df['Bearish'].plot(figsize=(17,8),legend=True, color='pink',label='ShortTermSellSignal')
df['MACD'].plot(figsize=(17,8), legend = True)
df['EMA9M'].plot(figsize=(17,8), legend = True)
plt.axhline(y=0, color='k', linestyle='-',label='CentreLine')
plt.grid(True);
plt.title('Centre line Indicator')

In [ ]:
df['Indicator']=[1 if df.loc[ei,'Buy']|df.loc[ei,'Sell'] else 0 for ei in df.index ]
df['Sum']=0
for j in range(1,11):
    df['Sum']+=df['Close'].shift(-j)-df['Close']
df['Mean_Return_NXT_10D']=abs(df['Sum']/(df['Close']*10))
df.tail(15)

In [ ]:
keep_df=df[['Close','Mean_Return_NXT_10D','Indicator']] 
#slices the DataFrame: to drop last 10 columns
final_df=keep_df[:-10].round({'Close':2,'Mean_Return_NXT_10D':3})
final_df